In [16]:
# Let's make sure these directories are clean before we start
import shutil
try:
    shutil.rmtree("./data/Food-cnn-split/train")
    shutil.rmtree("./data/Food-cnn-split/test")
except:
    pass

In [17]:
# We have three class which contains all the data: Bread, Soup and Vegetable-Fruit
# Let's create directories for each class in the train and test directories.
import os
# ensure directories exist
from pathlib import Path

Path("./data/Food-cnn-split/train/Bread").mkdir(parents=True, exist_ok=True)
Path("./data/Food-cnn-split/train/Soup").mkdir(parents=True, exist_ok=True)
Path("./data/Food-cnn-split/train/Vegetable-Fruit").mkdir(parents=True, exist_ok=True)

Path("./data/Food-cnn-split/test/Bread").mkdir(parents=True, exist_ok=True)
Path("./data/Food-cnn-split/test/Soup").mkdir(parents=True, exist_ok=True)
Path("./data/Food-cnn-split/test/Vegetable-Fruit").mkdir(parents=True, exist_ok=True)

In [18]:
# we need paths of images for individual classes so we can copy them in the new directories that we created above
all_bread_file_paths = os.listdir('./data/Food_Data/Bread')
all_soup_file_paths = os.listdir('./data/Food_Data/Soup')
all_vegetable_fruit_file_paths = os.listdir('./data/Food_Data/Vegetable-Fruit')

In [19]:
import random

train_bread_paths = random.sample(all_bread_file_paths, int(len(all_bread_file_paths)*0.8))
print("train bread image count: ", len(train_bread_paths))
test_bread_paths = [ p for p in all_bread_file_paths if p not in train_bread_paths]
print("test bread image count: ", len(test_bread_paths))
# ensure no overlap:
overlap = [p for p in train_bread_paths if p in test_bread_paths]
print("len of overlap: ", len(overlap))

train_soup_paths = random.sample(all_soup_file_paths, int(len(all_soup_file_paths)*0.8))
print("train soup image count: ", len(train_soup_paths))
test_soup_paths = [ p for p in all_soup_file_paths if p not in train_soup_paths]
print("test soup image count: ", len(test_soup_paths))
# ensure no overlap:
overlap = [p for p in train_soup_paths if p in test_soup_paths]
print("len of overlap: ", len(overlap))

train_vegetable_fruit_paths = random.sample(all_vegetable_fruit_file_paths, int(len(all_vegetable_fruit_file_paths)*0.8))
print("train vegetable fruit image count: ", len(train_vegetable_fruit_paths))
test_vegetable_fruit_paths = [ p for p in all_vegetable_fruit_file_paths if p not in train_vegetable_fruit_paths]
print("test vegetable fruitimage count: ", len(test_vegetable_fruit_paths))
# ensure no overlap:
overlap = [p for p in train_bread_paths if p in test_bread_paths]
print("len of overlap: ", len(overlap))

train bread image count:  241
test bread image count:  61
len of overlap:  0
train soup image count:  241
test soup image count:  61
len of overlap:  0
train vegetable fruit image count:  186
test vegetable fruitimage count:  47
len of overlap:  0


In [20]:
# ensure to copy the images to the directories
import shutil
for p in train_bread_paths:
    shutil.copyfile(os.path.join('./data/Food_Data/Bread', p), os.path.join('./data/Food-cnn-split/train/Bread', p) )

for p in test_bread_paths:
    shutil.copyfile(os.path.join('./data/Food_Data/Bread', p), os.path.join('./data/Food-cnn-split/test/Bread', p) )

for p in train_soup_paths:
    shutil.copyfile(os.path.join('./data/Food_Data/Soup', p), os.path.join('./data/Food-cnn-split/train/Soup', p) )

for p in test_soup_paths:
    shutil.copyfile(os.path.join('./data/Food_Data/Soup', p), os.path.join('./data/Food-cnn-split/test/Soup', p) )

for p in train_vegetable_fruit_paths:
    shutil.copyfile(os.path.join('./data/Food_Data/Vegetable-Fruit', p), os.path.join('./data/Food-cnn-split/train/Vegetable-Fruit', p) )

for p in test_vegetable_fruit_paths:
    shutil.copyfile(os.path.join('./data/Food_Data/Vegetable-Fruit', p), os.path.join('./data/Food-cnn-split/test/Vegetable-Fruit', p) )


# check counts:
print("Files in train/bread: ", len(os.listdir("./data/Food-cnn-split/train/Bread")))
print("Files in train/soup: ", len(os.listdir("./data/Food-cnn-split/train/Soup")))
print("Files in train/vegetable-fruit: ", len(os.listdir("./data/Food-cnn-split/train/Vegetable-Fruit")))

print("Files in test/bread: ", len(os.listdir("./data/Food-cnn-split/test/Bread")))
print("Files in test/soup: ", len(os.listdir("./data/Food-cnn-split/test/Soup")))
print("Files in test/vegetable-fruit: ", len(os.listdir("./data/Food-cnn-split/test/Vegetable-Fruit")))

Files in train/bread:  241
Files in train/soup:  241
Files in train/vegetable-fruit:  186
Files in test/bread:  61
Files in test/soup:  61
Files in test/vegetable-fruit:  47


In [21]:
import tensorflow as tf
import tensorflow_datasets as tfds
# Import Rescaling layer
from tensorflow.keras.layers import Rescaling
train_data_dir = 'data/Food-cnn-split/train'

batch_size = 32
# target image size
img_height = 150
img_width = 150

# note that subset="training", "validation", "both", and dictates which dataset is returned
train_ds, val_ds = tf.keras.utils.image_dataset_from_directory(
train_data_dir,
validation_split=0.2,
subset="both",
seed=123,
image_size=(img_height, img_width),
batch_size=batch_size
)
rescale = Rescaling(scale=1.0/255)
train_rescale_ds = train_ds.map(lambda image,label:(rescale(image),label))
val_rescale_ds = val_ds.map(lambda image,label:(rescale(image),label))

Found 666 files belonging to 3 classes.
Using 533 files for training.
Using 133 files for validation.


In [22]:
test_data_dir = 'data/Food-cnn-split/test/'

batch_size = 2

# this is what was used in the paper --
img_height = 150
img_width = 150

# note that subset="training", "validation", "both", and dictates what is returned
test_ds = tf.keras.utils.image_dataset_from_directory(
test_data_dir,
seed=123,
image_size=(img_height, img_width),
)

# approach 1: manually rescale data --
rescale = Rescaling(scale=1.0/255)
test_rescale_ds = test_ds.map(lambda image,label:(rescale(image),label))

Found 168 files belonging to 3 classes.


# **Any Regular CNN**

In [23]:
from keras import layers
from keras import models
import pandas as pd
from keras import optimizers

# Intializing a sequential model
model_cnn = models.Sequential()

# Adding first conv layer with 64 filters and kernel size 3x3 , padding 'same' provides the output size same as the input size
model_cnn.add(layers.Conv2D(64, (3, 3), activation='relu', padding="same", input_shape=(150,150,3)))

# Adding max pooling to reduce the size of output of first conv layer
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

model_cnn.add(layers.Conv2D(32, (3, 3), activation='relu', padding="same"))
model_cnn.add(layers.MaxPooling2D((2, 2), padding = 'same'))

# flattening the output of the conv layer after max pooling to make it ready for creating dense connections
model_cnn.add(layers.Flatten())

# Adding a fully connected dense layer with 100 neurons
model_cnn.add(layers.Dense(100, activation='relu'))

# Adding a fully connected dense layer with 84 neurons
model_cnn.add(layers.Dense(84, activation='relu'))

# Adding the output layer with * neurons and activation functions as softmax since this is a multi-class classification problem
model_cnn.add(layers.Dense(3, activation='softmax'))

# Compile model
# RMSprop (Root Mean Square Propagation) is commonly used in training deep neural networks.
model_cnn.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_cnn.summary()

Model: "sequential_3"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_5 (Conv2D)               │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_3 (MaxPooling2D)  │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_6 (Conv2D)               │ (None, 75, 75, 32)     │        18,464 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_4 (MaxPooling2D)  │ (None, 38, 38, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_7 (Conv2D)               │ (None, 38, 38, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_5 (MaxPooling2D)  │ (None, 19, 19, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_3 (Flatten)             │ (None, 11552)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 100)            │     1,155,300 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_10 (Dense)                │ (None, 84)             │         8,484 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_11 (Dense)                │ (None, 3)              │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,193,543 (4.55 MB)

 Trainable params: 1,193,543 (4.55 MB)

 Non-trainable params: 0 (0.00 B)

In [24]:
#fit the model from image generator
history = model_cnn.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=10,
            validation_data=val_rescale_ds
)

Epoch 1/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 22s 988ms/step - accuracy: 0.3492 - loss: 1.1076 - val_accuracy: 0.3609 - val_loss: 1.0964
Epoch 2/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 17s 918ms/step - accuracy: 0.3664 - loss: 1.0926 - val_accuracy: 0.3985 - val_loss: 1.0696
Epoch 3/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 17s 928ms/step - accuracy: 0.4152 - loss: 1.0749 - val_accuracy: 0.3233 - val_loss: 1.1056
Epoch 4/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 17s 903ms/step - accuracy: 0.4437 - loss: 1.0434 - val_accuracy: 0.3684 - val_loss: 1.0598
Epoch 5/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 15s 784ms/step - accuracy: 0.5330 - loss: 1.0008 - val_accuracy: 0.4737 - val_loss: 0.9947
Epoch 6/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 13s 692ms/step - accuracy: 0.5563 - loss: 0.9660 - val_accuracy: 0.4286 - val_loss: 0.9921
Epoch 7/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 16s 873ms/step - accuracy: 0.5754 - loss: 0.9134 - val_accuracy: 0.6842 - val_loss: 0.8746
Epoch 8/10
17/17 ━━━━━━━━━━━━━━━━━━━━ 13s 671ms/step - accuracy: 0.5973 - loss: 0.8651 - val_accu

In [25]:
test_loss, test_accuracy = model_cnn.evaluate(test_rescale_ds, verbose=0)
test_accuracy

0.5535714030265808

# **LeNet-5**

In [11]:
from keras import layers
from keras import models
import pandas as pd

model_lenet5 = models.Sequential()

# Layer 1: Convolutional layer with 6 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(6, kernel_size=(3, 3), activation='relu', input_shape=(150,150,3)))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Layer 2: Convolutional layer with 16 filters of size 3x3, followed by average pooling
model_lenet5.add(layers.Conv2D(16, kernel_size=(3, 3), activation='relu'))
model_lenet5.add(layers.AveragePooling2D(pool_size=(2, 2)))

# Flatten the feature maps to feed into fully connected layers
model_lenet5.add(layers.Flatten())

# Layer 3: Fully connected layer with 120 neurons
model_lenet5.add(layers.Dense(120, activation='relu'))

# Layer 4: Fully connected layer with 84 neurons
model_lenet5.add(layers.Dense(84, activation='relu'))

# Output layer: Fully connected layer with num_classes neurons (e.g., 3 )
model_lenet5.add(layers.Dense(3, activation='softmax'))

# Compile model
model_lenet5.compile(optimizer=optimizers.RMSprop(learning_rate=1e-4), loss='sparse_categorical_crossentropy', metrics=['accuracy'])

# Generating the summary of the model
model_lenet5.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d_3 (Conv2D)               │ (None, 148, 148, 6)    │           168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d               │ (None, 74, 74, 6)      │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_4 (Conv2D)               │ (None, 72, 72, 16)     │           880 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ average_pooling2d_1             │ (None, 36, 36, 16)     │             0 │
│ (AveragePooling2D)              │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 20736)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 120)            │     2,488,440 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 84)             │        10,164 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 3)              │           255 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 2,499,907 (9.54 MB)

 Trainable params: 2,499,907 (9.54 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
#fit the model from image generator
history = model_lenet5.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds
)

Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 5s 211ms/step - accuracy: 0.4363 - loss: 0.9791 - val_accuracy: 0.5765 - val_loss: 0.7296
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 179ms/step - accuracy: 0.5927 - loss: 0.7146 - val_accuracy: 0.5765 - val_loss: 0.6878
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 174ms/step - accuracy: 0.6147 - loss: 0.6858 - val_accuracy: 0.5647 - val_loss: 0.6530
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 186ms/step - accuracy: 0.6285 - loss: 0.6398 - val_accuracy: 0.6706 - val_loss: 0.6205
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 172ms/step - accuracy: 0.6982 - loss: 0.6001 - val_accuracy: 0.6941 - val_loss: 0.5995
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 2s 166ms/step - accuracy: 0.7314 - loss: 0.5856 - val_accuracy: 0.7176 - val_loss: 0.5806
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 172ms/step - accuracy: 0.7658 - loss: 0.5527 - val_accuracy: 0.6588 - val_loss: 0.5827
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 3s 183ms/step - accuracy: 0.7552 - loss: 0.5518 - val_accuracy: 0.

# **VGG16**

In [13]:
# Import VGG16 model from Keras applications
from keras.applications.vgg16 import VGG16

#Load the pre-trained VGG16 model with weights trained on ImageNet
vgg_model = VGG16(weights='imagenet', include_top = False, input_shape = (150,150,3))
vgg_model.summary()

# Making all the layers of the VGG model non-trainable. i.e. freezing them
for layer in vgg_model.layers:
    layer.trainable = False

# Initializing the model
new_model = models.Sequential()

# Adding the convolutional part of the VGG16 model from above
new_model.add(vgg_model)

# Flattening the output of the VGG16 model because it is from a convolutional layer
new_model.add(layers.Flatten())

# Adding a dense input layer
new_model.add(layers.Dense(32, activation='relu'))

# Adding dropout prevents overfitting
new_model.add(layers.Dropout(0.2))

# Adding second input layer
new_model.add(layers.Dense(32, activation='relu'))

# Adding output layer
new_model.add(layers.Dense(3, activation='softmax'))

# Compiling the model
new_model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])
# Summary of the model
new_model.summary()

#fit the model from image generator
history = new_model.fit(
            train_rescale_ds,
            batch_size=32,
            epochs=20,
            validation_data=val_rescale_ds,
)

test_loss, test_accuracy = new_model.evaluate(test_rescale_ds, verbose=0)

Model: "vgg16"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer_2 (InputLayer)      │ (None, 150, 150, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv1 (Conv2D)           │ (None, 150, 150, 64)   │         1,792 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_conv2 (Conv2D)           │ (None, 150, 150, 64)   │        36,928 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block1_pool (MaxPooling2D)      │ (None, 75, 75, 64)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv1 (Conv2D)           │ (None, 75, 75, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_conv2 (Conv2D)           │ (None, 75, 75, 128)    │       147,584 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block2_pool (MaxPooling2D)      │ (None, 37, 37, 128)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv1 (Conv2D)           │ (None, 37, 37, 256)    │       295,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv2 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_conv3 (Conv2D)           │ (None, 37, 37, 256)    │       590,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block3_pool (MaxPooling2D)      │ (None, 18, 18, 256)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv1 (Conv2D)           │ (None, 18, 18, 512)    │     1,180,160 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv2 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_conv3 (Conv2D)           │ (None, 18, 18, 512)    │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block4_pool (MaxPooling2D)      │ (None, 9, 9, 512)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv1 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv2 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_conv3 (Conv2D)           │ (None, 9, 9, 512)      │     2,359,808 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ block5_pool (MaxPooling2D)      │ (None, 4, 4, 512)      │             0 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 14,714,688 (56.13 MB)

 Non-trainable params: 0 (0.00 B)

...

 Total params: 14,714,688 (56.13 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 32s 3s/step - accuracy: 0.7388 - loss: 0.5747 - val_accuracy: 0.9529 - val_loss: 0.1741
Epoch 2/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 29s 3s/step - accuracy: 0.9513 - loss: 0.2254 - val_accuracy: 0.9765 - val_loss: 0.0945
Epoch 3/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 36s 3s/step - accuracy: 0.9850 - loss: 0.0511 - val_accuracy: 0.9882 - val_loss: 0.1042
Epoch 4/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 36s 3s/step - accuracy: 0.9835 - loss: 0.0378 - val_accuracy: 0.9882 - val_loss: 0.0910
Epoch 5/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 37s 3s/step - accuracy: 0.9940 - loss: 0.0220 - val_accuracy: 0.9882 - val_loss: 0.0993
Epoch 6/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 0.9871 - loss: 0.0289 - val_accuracy: 0.9882 - val_loss: 0.1093
Epoch 7/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 34s 3s/step - accuracy: 0.9935 - loss: 0.0214 - val_accuracy: 0.9882 - val_loss: 0.1164
Epoch 8/20
11/11 ━━━━━━━━━━━━━━━━━━━━ 0s 2s/step - accuracy: 1.0000 - loss: 0.0088

KeyboardInterrupt: 